In [1]:
# imports de bibliotecas
import os
import numpy as np

In [20]:
ARQUIVO_ANALEX = 'entrada.txt'
TABELA_SIMBOLOS = 'tabela.txt'

# Declaração de códigos de erro
ARQUIVO_INVALIDO = 1

# Declaração de todos os sinais e palavras reservadas da linguagem
sinais = ['+', '-', '*', '/', '^', '√']
palavras_reservadas = ['(', ')']

def carregarArquivo(arquivo):
  lista_diretorio = os.listdir()
  if arquivo in lista_diretorio:
    entrada = open(arquivo, 'r')
    return entrada
  else:
      return False

def tratarErros(erro):
  if erro == ARQUIVO_INVALIDO:
    print('Arquivo não encontrado.')

def analex(nome_arquivo):
  arquivo = carregarArquivo(nome_arquivo)
  if arquivo:
    print('Arquivo carregado!\n', arquivo)
  else:
    tratarErros(ARQUIVO_INVALIDO)
  # file.read(1)

  # recebe atomo
  # verifica se é número
  #   se for, adiciona ao token e repete classificacao
  # classifica como número
  # senão
  # verifica se é operador
  #   se for, classifica o token como operador
  # senão
  # verifica se é palavra reservada
  #   se for, classifica o token como palavra reservada

#resultado = carregarArquivo(ARQUIVO_ANALEX)
#print(resultado)
analex(ARQUIVO_ANALEX)

Arquivo carregado!
 <_io.TextIOWrapper name='entrada.txt' mode='r' encoding='UTF-8'>
